# Ownership

STACK 
- stack of plates, last in first out, all same size.
- pushing data onto stack is not allocating
- Pushing to stack faster than heap - no need to search for space big enough, the location is simply always on top of stack

HEAP 
- data of unknown size at compile time or size that might change is stored on the heap.
- when you request certain mem size, mem allocator finds space big enough and returns a pointer - "allocating on the heap"
- pointer to the heap is known, of fixed size, you can store it on a stack -> if you want the data you have to go on heap
- 


ex. waiting for a place in restaurant:
- you state the number of ppl, host finds table big enough, if someone from my group comes late they can ask where they seated you.

Processors are faster if they jump around less in memory.
- its faster if server first takes all orders from table A, then from table B etc. than jumping from A to B
- PROCESSOR CAN DO ITS JOB BETTER IF IT WORKS ON DATA THATS CLOSE TO OTHER DATA
- the other data is on STACK already, rather than farther away (as that can be on the HEAP)

<br >
CALLING A FUNC

- when code calls a function, all data entering the function, (including possible pointers to the heap) are pushed onto STACK

- when function is over, those vals get popped of the STACK

OBJECTIVES
- keeping track of what parts of code is accessing what data on the heap
- minimize duplicate data on heap
- cleaning up unused data on heap, so we dont run out.
- main purpose of ownership is to manage heap data


#### Ownership
- each value has an owner
- there can be only one owner at a time
- when the owner goes out of scope, values get dropped

In [2]:
{
    let s = "hello";
}

()

- i32, i64 ... known size, can be on STACK and popped off when their scope is over
- String... stored on HEAP, cause size unknown
- String literal type immutable, second string type can be mutable ...HEAP


In [8]:
fn freeing_mem() {
    {
        let mut s = String::from("hello");
        s.push_str(", world!");
        println!("{s}");
    }

    println!("{s}");
}

Error: cannot find value `s` in this scope

- str literal we known contents at compile time, text is hardcoded into final executable
- String type, we need growable piece of text hence amount of mem of heap, unknown at compile time
- String::from requests memory it needs
- We need a way of returning this memory to the alloc when we are done with String. How ?
    - GC - Garbage collector
    - Freeing memory is our responsibility
        - Difficult problem to free mem on our own
        - pairing exactly one **allocate** with one **free**
    - Rust returns memory once the variable that owns it goes out of scope.
    - Rust calls drop function automatically at closing curly bracket

#### Moving variables

In [12]:
// bind val 5 to x, make a copy of val in x and bind to y
// integers are simple vals of fixed size that can be pushed onto the stack
let x = 5;
let y = x;

Strings are made of pointer on the stack and memory allocated on the heap

In [17]:
// string is different
// s1: STACK: [pointer, len, capacity] -> HEAP [index, value] 
// len how much mem in bytes the string is currently using
// capacity is total amt of mem that String has received from the allocator
let s1 = String::from("hello");
let s2 = s1;

So what happens above ? 
- assigning s1 to s2 does copy the STACK part: [pointer, len, capacity], but does nothin with the HEAP part.
- effectively creating two pointers to the same heap memory space

double free error
- if both s1 and s2 go out of scope, they both try to free the same memory, this is true for s1, s2 poiting to HEAP

In [ ]:
fn prevents_double_free() {
    // s1 does not implement the copy trait
    let s1 = String::from("Hello");
    // moving s1 into s2, MOVE / "shallow copy" only the STACK pointers
    let s2 = s1;
    // s1 no longer valid
    println!("{s1}")
}
prevents_double_free()

// rust never creates deep copies on its own.

Error: unused variable: `s2`

Error: borrow of moved value: `s1`

In [27]:
fn overwriting_strings() {
    // get STACK [pointer, len , capacity] -> HEAP [index, val]
    let mut s = String::from("hello");
    // overwrite the HEAP part only
    s = String::from("ahoj");

    // original string goes out of scope, Rust calls .drop

    println!("{s}, world!");
}
overwriting_strings()

ahoj, world!


()

In [32]:
fn cloning_is_deep_copy() {
    let s1 = String::from("hello");
    let s2 = s1.clone();
    
    println!("{s1}, {s2}");
}
cloning_is_deep_copy()

hello, hello


()

#### Stack only data

In [36]:
fn fixed_sized_does_deep_copy() {
    // i32 fixed size, copy easy to make
    let x: i32 = 5;
    let y: i32 = x;
    println!("{x}, {y}");
}

fixed_sized_does_deep_copy()

5, 5


()

#### Copy annotation

In [46]:
fn takes_ownership(some_string: String) {
    // String unknown size at compile time so MOVE
    println!("{some_string}");
}

fn makes_copy(some_int: i32) {
    // i32 known size at compile time, STACK only, so COPY
    println!("{some_int}");
}

fn moving_is_ownership() {
    let s = String::from("ahoy");

    takes_ownership(s); // s moves into fn
                        // s no longer valid here

    // println!("{s}"); // s got dropped this fails


    let x:i32 = 5;      // x comes into scope
    makes_copy(x);      // i32 implements Copy trait so x does NOT move

    println!("{x}");
}

moving_is_ownership();

ahoy
5
5


types implementing Copy trait
- u32, i32 ...
- bool
- f64
- char
- (i32, i32) Tuples if they only contain types that impement Copy
- (i32, String) does not implement Copy

#### Returning values and scope

In [ ]:
fn gives_ownership() -> String {
    let some_string = String::from("ahoy is yours");

    return some_string;
}

fn takes_and_gives_back(a_string: String) -> String {
    return a_string;
}

fn return_vals_and_scope() { 
    let s1 = gives_ownership();

    let s2 = String::from("ETELA");
    let s3 = takes_and_gives_back(s2);

    // println!("{s2}"); FAILS cause s2 is moved into s3
}

return_vals_and_scope();

In [55]:
fn returning_tuples() {
    let s1 = String::from("hello");

    let (s2, len) = calculate_length(s1);

    println!("The length of '{s2}' is {len}.");
}

fn calculate_length(s: String) -> (String, usize) {
    let length = s.len(); // len() returns the length of a String

    (s, length)
}

returning_tuples();

The length of 'hello' is 5.
